# BundesFAQ RAG System mit Semantic Chunking

Dieses Notebook implementiert ein Retrieval-Augmented Generation System für deutsche FAQ-Daten mit semantischem Chunking und ChromaDB.

## Environment Setup

Laden der Umgebungsvariablen und Überprüfung der OpenAI API Key Verfügbarkeit.

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
print("API Key gefunden:", "OPENAI_API_KEY" in os.environ)

API Key gefunden: True


## Daten laden

FAQ-Textdatei aus dem data Verzeichnis laden und ersten Überblick anzeigen.

In [2]:
from pathlib import Path

file_path = Path("../../data/FAQtxt.txt")
text = file_path.read_text(encoding="utf-8")

print(f"Datei geladen: {len(text)} Zeichen")
print(text[:500])

Datei geladen: 26510 Zeichen
Frage: Gibt es einen Rechtsanspruch auf Daten?
Antwort: Weder besteht nach § 12a Abs. 1 Satz 2 EGovG ein Rechtsanspruch auf Daten, noch nach § 2a IWG ein Rechtsanspruch auf Informationen. Laut § 1 IFG hat jeder gegenüber Behörden einen Anspruch auf Zugang zu amtlichen Informationen. Im Gegensatz zu § 12a EGovG besteht jedoch ein Antragserfordernis: Bürgerinnen und Bürger müssen eine sogenannte IFG-Anfrage stellen, um ihr rechtliches oder berechtigtes Interesse geltend zu machen.

Frage: Kann der


## Semantic Chunking

Text in semantische Chunks aufteilen basierend auf "Frage:" Pattern. Jeder Chunk wird als Document Objekt gespeichert.

In [3]:
import re
from langchain.docstore.document import Document

faq_blocks = re.split(r"\n(?=Frage:)", text)

chunks = []
for block in faq_blocks:
    block = block.strip()
    if block:
        chunks.append(Document(page_content=block, metadata={"source": "FAQ"}))

print(f"Gefundene Chunks: {len(chunks)}")
print("\n" + "="*80)

for i, chunk in enumerate(chunks[:4], 1):
    print(f"\nCHUNK {i}:")
    print("-" * 40)
    print(chunk.page_content[:500]) 
    print("="*80)

Gefundene Chunks: 42


CHUNK 1:
----------------------------------------
Frage: Gibt es einen Rechtsanspruch auf Daten?
Antwort: Weder besteht nach § 12a Abs. 1 Satz 2 EGovG ein Rechtsanspruch auf Daten, noch nach § 2a IWG ein Rechtsanspruch auf Informationen. Laut § 1 IFG hat jeder gegenüber Behörden einen Anspruch auf Zugang zu amtlichen Informationen. Im Gegensatz zu § 12a EGovG besteht jedoch ein Antragserfordernis: Bürgerinnen und Bürger müssen eine sogenannte IFG-Anfrage stellen, um ihr rechtliches oder berechtigtes Interesse geltend zu machen.

CHUNK 2:
----------------------------------------
Frage: Kann der Zugriff auf Daten eingeschränkt werden?
Antwort: Laut § 12a EGovG müssen die zu veröffentlichenden Daten ohne Einschränkung für jede Person zum Datenabruf über öffentlich zugängliche Netze bereitgestellt werden. Zudem muss die Abfrage entgeltfrei und zur uneingeschränkten Weiterverwendung möglich sein. In diesem Sinne ist auch die Lizenz so zu wählen, dass die Daten möglich

## ChromaDB Vektordatenbank

Erstellung der Vektordatenbank mit OpenAI Embeddings für semantische Suche.

In [4]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

persist_directory = "./chroma_db"
embedding = OpenAIEmbeddings(model="text-embedding-3-small")

print(f"Verfügbare Chunks: {len(chunks)}")
print(f"Erstelle Vektordatenbank...")

vectorstore = Chroma.from_documents(
    documents=chunks,  
    embedding=embedding,
    collection_name="bundesfaq_rag_collection",
    persist_directory=persist_directory,
)

print("Vektordatenbank erfolgreich erstellt!")
print(f"Anzahl Vektoren: {vectorstore._collection.count()}")
print(f"Gespeichert in: {persist_directory}")

# Statistik
total_chars = sum(len(chunk.page_content) for chunk in chunks)
avg_chars = total_chars / len(chunks) if chunks else 0
print(f"Durchschnittliche Chunk-Größe: {avg_chars:.0f} Zeichen")

Verfügbare Chunks: 42
Erstelle Vektordatenbank...


: 

## Similarity Search Test

Einfacher Test der Vektordatenbank mit einer Beispielfrage.

In [ ]:
query = "Was ist GovData.de?"  
docs = vectorstore.similarity_search(query, k=2)

print("Frage:", query)
print("\nTop Ergebnisse:")

for i, doc in enumerate(docs, 1):
    print(f"\n--- Ergebnis {i} ---")
    print(doc.page_content[:500]) 

## Detaillierte Suchergebnisse

Similarity Search mit Scores in tabellarischer Darstellung für bessere Analyse.

In [8]:
import pandas as pd
query = "Was ist GovData.de?"
results = vectorstore.similarity_search_with_score(query, k=5)

rows = []
for i, (doc, score) in enumerate(results, 1):
    parts = doc.page_content.split("Antwort:", 1)
    frage = parts[0].replace("Frage:", "").strip()
    antwort = parts[1].strip() if len(parts) > 1 else ""

    rows.append({
        "Rank": i,
        "Frage": frage,
        "Antwort (gekürzt)": antwort[:200] + "..." if len(antwort) > 200 else antwort,
        "Score": f"{score:.4f}",
        "Chunk Länge": len(doc.page_content)
    })

df = pd.DataFrame(rows)
df

,Rank,Frage,Antwort (gekürzt),Score,Chunk Länge
0,1,Was ist GovData.de?,GovData.de ist das zentrale Metadatenportal fü...,0.3440,749
1,2,Wo können die Daten veröffentlicht werden?,Die eigentlichen Daten sind frei zugänglich oh...,0.7074,886
2,3,Müssen die Daten auf GovData.de regelmäßig gep...,Bestandsdaten sollten regelmäßig von den Open-...,0.7199,331
3,4,Wo sind Metadaten zu veröffentlichen?,Gemäß § 12a EGovG sind offene Verwaltungsdaten...,0.7650,274
4,5,Was versteht man unter „Veröffentlichende Stel...,"Damit ist die Organisationseinheit gemeint, di...",0.7687,428


## RAG Chain Setup

Import der erforderlichen LangChain Komponenten für Retrieval-Augmented Generation.

In [9]:
from langchain_openai import ChatOpenAI
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

## LLM Initialisierung

ChatOpenAI Modell für die Antwortgenerierung konfigurieren.

In [10]:
llm = ChatOpenAI(model="gpt-3.5-turbo")  
print("LLM initialisiert")

LLM initialisiert


## Prompt Template

System Prompt für kontextbasierte Antwortgenerierung definieren.

In [ ]:
system_prompt = """You are a helpful assistant for question-answering tasks.  
Use the retrieved context below to answer the user's question.  

- If the answer is not in the context, say: "I don't know based on the provided documents."  
- Be concise (max. 3 sentences).  
- Ground your answer in the context, don't invent facts.  

Context:
{context}"""

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}")
])

## Document Chain

Verbindung zwischen LLM und Prompt Template erstellen.

In [12]:
document_chain = create_stuff_documents_chain(llm, prompt)

## Retriever Setup

Retriever aus der Vektordatenbank für relevante Dokumente konfigurieren.

In [13]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})


## RAG Chain

Vollständige RAG Kette aus Retriever und Document Chain zusammenfügen.

In [14]:
rag_chain = create_retrieval_chain(retriever, document_chain)


## RAG System Test

Test des vollständigen RAG Systems mit einer Beispielfrage.

In [15]:
response = rag_chain.invoke({"input": "Wie können die eigentlichen Daten mit Metadaten verknüpft werden?"})
print("Antwort:", response["answer"])

Antwort: Die eigentlichen Daten können mit Metadaten verknüpft werden, indem beschreibende Informationen wie Autor, Erscheinungsdatum, Stichwörter und Kontaktdaten im Metadatensatz gesammelt und mit den entsprechenden Daten verknüpft werden. Dadurch lassen sich die Daten bestimmten Typs oder Inhalts zielsicher und schnell finden.
